In [ ]:
class car:
    __brand =None
    def __init__(self,color,brand):
        self.color = color
        self.__brand = brand
    def ss(self):
        print(self.__brand)
       
c = car('blue','BMW')
print(c.color)
print(c.ss())

None
None


In [28]:
import numpy as np
s=np.array([1,1,1])
s2=np.array([[2,7,2],[2,2,2]])
np.unravel_index(np.argmax(s2), s2.shape) 

(0, 1)

In [31]:
class L2Loss:
    def __init__(self):
        self.input_tensor = None
    def forward(self, input_tensor, label_tensor):
        self.input_tensor = input_tensor
        return np.sum(np.square(input_tensor - label_tensor))
    def backward(self, label_tensor):
        return 2*np.subtract(self.input_tensor, label_tensor)
def gradient_check(layers, input_tensor, label_tensor):
    epsilon = 1e-5
    difference = np.zeros_like(input_tensor)

    activation_tensor = input_tensor.copy()
    for layer in layers[:-1]:
        activation_tensor = layer.forward(activation_tensor)
    layers[-1].forward(activation_tensor, label_tensor)

    error_tensor = layers[-1].backward(label_tensor)
    for layer in reversed(layers[:-1]):
        error_tensor = layer.backward(error_tensor)

    it = np.nditer(input_tensor, flags=['multi_index'])
    while not it.finished:
        plus_epsilon = input_tensor.copy()
        plus_epsilon[it.multi_index] += epsilon
        minus_epsilon = input_tensor.copy()
        minus_epsilon[it.multi_index] -= epsilon

        analytical_derivative = error_tensor[it.multi_index]

        for layer in layers[:-1]:
            plus_epsilon = layer.forward(plus_epsilon)
            minus_epsilon = layer.forward(minus_epsilon)
        upper_error = layers[-1].forward(plus_epsilon, label_tensor)
        lower_error = layers[-1].forward(minus_epsilon, label_tensor)

        numerical_derivative = (upper_error - lower_error) / (2 * epsilon)

        # print('Analytical: ' + str(analytical_derivative) + ' vs Numerical :' + str(numerical_derivative))
        normalizing_constant = max(np.abs(analytical_derivative), np.abs(numerical_derivative))

        if normalizing_constant < 1e-15:
            difference[it.multi_index] = 0
        else:
            difference[it.multi_index] = np.abs(analytical_derivative - numerical_derivative) / normalizing_constant

        it.iternext()
    return difference

In [63]:
import math
import numpy as np
from Layers.Base import BaseLayer
class Conv(BaseLayer):
    def __init__(self, stride_shape, convolution_shape, num_kernel):
        self.trainable = True
        self.kernel_shape = (num_kernel, *convolution_shape)
        self.weights = np.random.uniform(0, 1, self.kernel_shape)
        self.bias = np.random.uniform(0, 1, (num_kernel, 1)) #(4,1)
        self.stride = stride_shape
        self.num_kernel = num_kernel
        self.convolution_shape = convolution_shape
        self.gw = None
        self.gb = None
        self.optimizer =None
        

    @property
    def gradient_weights(self):
        return self.gw

    @property
    def gradient_bias(self):
        return self.gb

    def initialize(self,weights_initializer_methods, bias_initializer_methods):
        fan_in = math.prod(self.convolution_shape)
        fan_out = fan_in / self.convolution_shape[0] * self.num_kernel
        self.weights = weights_initializer_methods.initialize(self.weights.shape,fan_in,fan_out)
        self.bias = bias_initializer_methods.initialize(self.bias.shape,fan_in,fan_out)

    def conv_1D(self, input, output_shape,stride):
        """
            input: (c, y)
            output_shape: (num_kernel, (y-m)/s)
            output: (num_kernel, (y-m)/s)
        """
        num_kernel, output_width = output_shape
        output = np.zeros(output_shape)

        for k in range(num_kernel):
            kernel = self.weights[k]
            bias = self.bias[k]
            for i in range(0, input.shape[-1] - kernel.shape[-1] + 1, stride):
                region = input[:, i:i + kernel.shape[-1]]
                output[k, i // stride] = np.sum(region * kernel) + bias

        return output

    def conv_2D(self, input, output_shape, strides):
        """
            input: (c, y, x)
            output_shape: (num_kernel, (y-m)/s_y, (x-n)/s_x)
            strides: (stride_y, stride_x)
            kernel: (num_kernel, c, y, x)
            output: (num_kernel, (y-m)/s_y, (x-n)/s_x)
        """
        stride_y, stride_x = strides
        num_kernel, output_height, output_width = output_shape
        output = np.zeros(output_shape)

        for k in range(num_kernel):
            kernel = self.weights[k]
            bias = self.bias[k]
            for i in range(0, input.shape[1] - kernel.shape[1]+1, stride_y):
                for j in range(0, input.shape[2] - kernel.shape[2]+1, stride_x):
                    region = input[:, i:i + kernel.shape[1], j:j + kernel.shape[2]]
                    output[k, i // stride_y, j // stride_x] = np.sum(region * kernel) + bias
        return output

    def forward(self, input_tensor):
        """
        input_tensor: (b, c, y) / (b, c, y, x)
        output_tensor: (b, num_kernel, (y-m)//s+1) /(b, num_kernel, (y-m)//s_y+1, (x-n)//s_x+1)
        """
        n = len(input_tensor.shape)
        output = []
        self.input_tensor = input_tensor
        if n == 3:
            # 1D convolution
            b, c, y = input_tensor.shape
            py = (self.kernel_shape[2]-1)//2
            py_end = py
            if self.kernel_shape[2]%2 ==0 :
                    py_end +=1
            self.padding = [(0,0),  (0,0), (py,py_end)]
            self.pad_input_tensor = np.pad(input_tensor, self.padding, mode='constant', constant_values=0)
            output_shape = (self.num_kernel, (self.pad_input_tensor.shape[-1] - self.kernel_shape[2]) // self.stride[0]+1)
            for i in range(b):
                single_input = self.pad_input_tensor[i]
                output.append(self.conv_1D(single_input, output_shape,self.stride[0]))
        elif n == 4:
            # 2D convolution
            b, c, y, x = input_tensor.shape
            if isinstance(self.stride, tuple):
                self.stride_y, self.stride_x = self.stride
            else:
                self.stride_y, self.stride_x = self.stride, self.stride
            px,py = (self.kernel_shape[3]-1)//2,(self.kernel_shape[2]-1)//2
            px_end,py_end = px,py
            if self.kernel_shape[3]%2 ==0 :
                    px_end +=1
            if  self.kernel_shape[2]%2 ==0 :
                    py_end +=1
            self.padding = [(0, 0), (0, 0), (py, py_end), (px, px_end)]
            self.pad_input_tensor = np.pad(input_tensor, self.padding, mode='constant', constant_values=0)

            output_shape = (self.num_kernel,
                            (self.pad_input_tensor.shape[-2]-self.weights.shape[-2])//self.stride_y+1,
                            (self.pad_input_tensor.shape[-1]-self.weights.shape[-1])//self.stride_x+1)
            for i in range(b):
                single_input = self.pad_input_tensor[i]
                output.append(self.conv_2D(single_input, output_shape, [self.stride_y, self.stride_x]))

        self.output = np.array(output)
        return self.output
    
    def backward_1D(self, error_tensor):
        db = np.sum(error_tensor, axis=(0,2)).reshape(-1,1)
        dw = np.zeros(self.weights.shape)
        padded_err = np.pad(error_tensor, [(0, 0), (0, 0), (self.kernel_shape[-1] - 1, self.kernel_shape[-1] - 1)], mode='constant', constant_values=0)
        for k in range(self.num_kernel):
            for i in range(0,(self.pad_input_tensor.shape[-1]-error_tensor.shape[-1] )//self.stride[0]-1):
                dw[k,:,i] = np.sum(self.pad_input_tensor[:,:,i:i+error_tensor.shape[2]] * error_tensor[:,k:k+1,:],axis=(0,2))
        self.gb = db
        self.gw = dw
        d_input = np.zeros(self.input_tensor.shape)#(b,c,w)
        flipped_weights=np.flip(self.weights, axis=-1)
        for n in range(self.input_tensor.shape[0]):  # 遍历批次
            for c_in in range(self.input_tensor.shape[1]):  # 遍历输入通道
                for c_out in range(self.weights.shape[0]):  # 遍历输出通道
                    # 卷积核与步幅处理
                    flipped_kernel = flipped_weights[c_out, c_in, :]
                    for i in range(error_tensor.shape[-1]):  # 按照步幅更新 d_input
                        start = i * self.stride[0]
                        end = start + self.weights.shape[-1]
                        if end <= self.input_tensor.shape[-1]:  # 确保范围合法
                            d_input[n, c_in, start:end] += error_tensor[n, c_out, i] * flipped_kernel
        return d_input
    
    def conv2d_backward(self,error_tensor, input_tensor, kernel, stride_y, stride_x, padding):
        # 获取各种维度信息
        batch_size, in_channel, input_height, input_width = input_tensor.shape
        num_kernels, _, kernel_height, kernel_width = kernel.shape

        # 计算填充后的输入维度
        padded_height = input_height + 2 * padding
        padded_width = input_width + 2 * padding
        pad_input_tensor = np.pad(input_tensor, ((0, 0), (0, 0), (padding, padding), (padding, padding)), 'constant')

        # 初始化梯度张量
        grad_input = np.zeros_like(pad_input_tensor)

        # 反向传播计算梯度
        for n in range(batch_size):
            for k in range(num_kernels):
                for c in range(in_channel):
                    for i in range(error_tensor.shape[2]):
                        for j in range(error_tensor.shape[3]):
                            h_start = i * stride_y
                            w_start = j * stride_x
                            h_end = h_start + kernel_height
                            w_end = w_start + kernel_width

                            grad_input[n, c, h_start:h_end, w_start:w_end] += kernel[k, c, :, :] * error_tensor[n, k, i, j]

        # 移除填充
        if padding > 0:
            grad_input = grad_input[:, :, padding:-padding, padding:-padding]

        return grad_input

    def backward_2D(self, error_tensor):
        """
        """
        db = np.sum(error_tensor, axis=(0,2,3)).reshape(-1,1)
        dw = np.zeros_like(self.weights)
        grad_input = np.zeros_like(self.pad_input_tensor)
        rotated_kernels = np.flip(self.weights, axis=(2, 3))
        # 卷积核梯度计算
        for n in range(error_tensor.shape[0]):  # 遍历批次
            for k in range(self.num_kernel):  # 遍历每个卷积核
                for c in range(self.kernel_shape[1]):  # 遍历输入通道
                    for i in range(error_tensor.shape[2]):  # 遍历输出高度
                        for j in range(error_tensor.shape[3]):  # 遍历输出宽度
                            start_i,end_i = i * self.stride_y,i * self.stride_y + self.kernel_shape[2]
                            start_j,end_j = j * self.stride_x,j * self.stride_x + self.kernel_shape[3]

                            if end_i <= self.pad_input_tensor.shape[2] and end_j <= self.pad_input_tensor.shape[3]:
                                input_slice = self.pad_input_tensor[n, c, start_i:end_i, start_j:end_j]
                                error_value = error_tensor[n, k, i, j]
                                dw[k, c, :, :] += input_slice * error_value
        """
        print(error_tensor.shape)
        print(self.kernel_shape)
        print(self.weights.shape)
        print(self.pad_input_tensor.shape)
        print(self.input_tensor.shape)
        print(self.padding[3])
        """
        # 输入梯度计算
        padded_error = np.pad(
            error_tensor,
            [(0, 0), (0, 0), (self.kernel_shape[2] - 1, self.kernel_shape[2] - 1), (self.kernel_shape[3] - 1, self.kernel_shape[3] - 1)],
            mode="constant",
            constant_values=0,
        )
        # 反向传播计算梯度
        for n in range(error_tensor.shape[0]):
            for k in range(self.num_kernel):
                for c in range(self.kernel_shape[1]):
                    for i in range(error_tensor.shape[2]):
                        for j in range(error_tensor.shape[3]):
                            h_start = i * self.stride_y
                            w_start = j * self.stride_x
                            h_end = h_start + self.kernel_shape[2]
                            w_end = w_start + self.kernel_shape[3]

                            grad_input[n, c, h_start:h_end, w_start:w_end] += self.weights[k, c, :, :] * error_tensor[n, k, i, j]
        # 去除填充部分，恢复输入形状
        py, py_end = self.padding[2]
        px, px_end = self.padding[3]
        grad_input = grad_input[:, :, py:-py_end if py_end != 0 else None, px:-px_end if px_end != 0 else None]
        
        #grad_input = self.conv2d_backward(error_tensor, self.input_tensor, self.weights, self.stride_y, self.stride_x, 1)
        # 保存梯度
        self.gb = db
        self.gw = dw
        return grad_input

    def backward(self, error_tensor):
        """
            error_tensor: (b, num_kernel, output_height, output_width) / (b, num_kernel, output_length)
        """
        if len(self.input_tensor.shape) == 3:  # 1D convolution backward
            gradin_input =  self.backward_1D(error_tensor)
        elif len(self.input_tensor.shape) == 4:  # 2D convolution backward
            gradin_input =  self.backward_2D(error_tensor)
        if self.optimizer is not None:
            self.weights = self.optimizer.calculate_update(self.weights,self.gw)
            self.bias = self.optimizer.calculate_update(self.bias,self.gb)
        return gradin_input

In [65]:
import unittest
from Optimization import Optimizers
from Layers import Initializers

class TestConv(unittest.TestCase):
    plot = False
    directory = 'plots/'

    class TestInitializer:
        def __init__(self):
            self.fan_in = None
            self.fan_out = None

        def initialize(self, shape, fan_in, fan_out):
            self.fan_in = fan_in
            self.fan_out = fan_out
            weights = np.zeros((1, 3, 3, 3))
            weights[0, 1, 1, 1] = 1
            return weights

    def setUp(self):
        self.batch_size = 2
        self.input_shape = (3, 10, 14)
        self.input_size = 14 * 10 * 3
        self.uneven_input_shape = (3, 11, 15)
        self.uneven_input_size = 15 * 11 * 3
        self.spatial_input_shape = np.prod(self.input_shape[1:])
        self.kernel_shape = (3, 5, 8)
        self.num_kernels = 4
        self.hidden_channels = 3

        self.categories = 105
        self.label_tensor = np.zeros([self.batch_size, self.categories])
        for i in range(self.batch_size):
            self.label_tensor[i, np.random.randint(0, self.categories)] = 1
    def test_update(self):
        input_tensor = np.random.uniform(-1, 1, (self.batch_size, *self.input_shape))
        conv = Conv((3, 2), self.kernel_shape, self.num_kernels)
        conv.optimizer = Optimizers.Sgd(1)
        conv.initialize(Initializers.He(), Initializers.Constant(0.1))
        # conv.weights = np.random.rand(4, 3, 5, 8)
        # conv.bias = 0.1 * np.ones(4)
        for _ in range(10):
            output_tensor = conv.forward(input_tensor)
            error_tensor = np.zeros_like(output_tensor)
            error_tensor -= output_tensor
            conv.backward(error_tensor)
            new_output_tensor = conv.forward(input_tensor)
            print(np.sum(np.power(output_tensor, 2)), np.sum(np.power(new_output_tensor, 2)))
test = TestConv()
test.setUp()
test.test_update()

104.87743662253294 286940.8973849519
286940.8973849519 1303569929.1050885
1303569929.1050885 6953521110109.354
6953521110109.354 3.993451309537277e+16
3.993451309537277e+16 2.3958568256211694e+20
2.3958568256211694e+20 1.481805749509826e+24
1.481805749509826e+24 9.382710787347455e+27
9.382710787347455e+27 6.056300202804526e+31
6.056300202804526e+31 3.9727468387244e+35
3.9727468387244e+35 2.6419073227920248e+39


C:\Users\HBen_\AppData\Local\Temp\ipykernel_30208\2690429956.py:68: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  output[k, i // stride_y, j // stride_x] = np.sum(region * kernel) + bias


In [2]:
import numpy as np
import copy
class NeuralNetwork():
    def __init__(self,optimizer,Initial_weights_method,Initial_bias_method):
        self.optimizer=optimizer
        self.Initial_weights_method=Initial_weights_method
        self.Initial_bias_method=Initial_bias_method
        self.loss=[]
        self.layers=[]
        self.data_layer=None
        self.loss_layer=None
        self.label=None
        self.output=None
    def forward(self):
        input_tensor,label_tensor=self.data_layer.next()
        self.label=label_tensor
        output_tensor = input_tensor.copy()
        for layer in self.layers:
            output_tensor = layer.forward(output_tensor)
        self.output=self.loss_layer.forward(output_tensor,self.label)
        return self.output
    def backward(self):
        error_tensor = self.loss_layer.backward(self.label)
        for layer in reversed(self.layers):
            error_tensor = layer.backward(error_tensor)
    def append_layer(self,layer):
        if layer.trainable:
            layer.initialize(self.Initial_weights_method,self.Initial_bias_method)
            layer.optimizer = copy.deepcopy(self.optimizer)
        self.layers.append(layer)

    def train(self,iterations):
        for it in range(iterations):
            self.loss.append(self.forward())
            self.backward()
    def test(self,input_tensor):
        output_tensor = input_tensor.copy()
        for layer in self.layers:
            output_tensor = layer.forward(output_tensor)
        return output_tensor

In [3]:
from Optimization import *
from Layers import *
def test_iris_data_with_adam():
        net = NeuralNetwork(Optimizers.Adam(1e-2, 0.9, 0.999),
                                          Initializers.UniformRandom(),
                                          Initializers.Constant(0.1))
        categories = 3
        input_size = 4
        net.data_layer = Helpers.IrisData(100)
        net.loss_layer = Loss.CrossEntropyLoss()
        fcl_1 = FullyConnected.FullyConnected(input_size, categories)
        net.append_layer(fcl_1)
        net.append_layer(SoftMax.SoftMax())
        net.train(20)
        data, labels = net.data_layer.get_test_set()

        results = net.test(data)
        accuracy = Helpers.calculate_accuracy(results, labels)
        print(accuracy, 0.9)

test_iris_data_with_adam()

-------------------------------
(5, 3)
(5, 3)
Adam: _____ start(4, 3)
Adam:(4, 3)
(4, 3) (4, 3)
(4, 3)
(4, 3)
(4, 3)
Adam: _____ end(4, 3)
Adam: _____ start(1, 3)
Adam:(1, 3)
(1, 3) (1, 3)
(1, 3)
(1, 3)
(1, 3)
Adam: _____ end(1, 3)
-------------------------------
(5, 3)
(5, 3)
Adam: _____ start(4, 3)
Adam:(4, 3)
(4, 3) (4, 3)
(4, 3)
(4, 3)
(4, 3)
Adam: _____ end(4, 3)
Adam: _____ start(1, 3)
Adam:(1, 3)
(1, 3) (1, 3)
(1, 3)
(1, 3)
(1, 3)
Adam: _____ end(1, 3)
-------------------------------
(5, 3)
(5, 3)
Adam: _____ start(4, 3)
Adam:(4, 3)
(4, 3) (4, 3)
(4, 3)
(4, 3)
(4, 3)
Adam: _____ end(4, 3)
Adam: _____ start(1, 3)
Adam:(1, 3)
(1, 3) (1, 3)
(1, 3)
(1, 3)
(1, 3)
Adam: _____ end(1, 3)
-------------------------------
(5, 3)
(5, 3)
Adam: _____ start(4, 3)
Adam:(4, 3)
(4, 3) (4, 3)
(4, 3)
(4, 3)
(4, 3)
Adam: _____ end(4, 3)
Adam: _____ start(1, 3)
Adam:(1, 3)
(1, 3) (1, 3)
(1, 3)
(1, 3)
(1, 3)
Adam: _____ end(1, 3)
-------------------------------
(5, 3)
(5, 3)
Adam: _____ start(4, 3)
Ad

In [3]:
arr = np.random.random((4, 3))
print(arr.shape)
a=Optimizers.Adam(5e-3, 0.98, 0.999)
a.calculate_update(arr[-1:,:],arr[-1:,:]).shape

(4, 3)
Adam: _____ start(1, 3)
Adam:(1, 3)
(1, 3)
(1, 3)
Adam: _____ end(1, 3)


(1, 3)